Árboles de decisión
================

- Uno de los métodos más utilizados.

- Fáciles y rápidos de construir.

- Modelo no-paramétrico: vamos a definir un montón de hiper-parámetros pero el modelo en sí no va a tener pesos ni nada de eso.

- **White-box model**: permiten inferir conocimiento de qué es lo que está pasando.

- El proceso de construcción del árbol realiza un filtrado de variables (estilo embebbed).

- El árbol puede ser expresado como un conjunto de reglas "if-then".

- Podemos utilizarlos para clasificación y regresión.

(Inicialmente nos vamos a concentrar en problemas de clasificación)

# Cuándo?

- Me interesa dar explicaciones de las salidas que da el modelo
- Prototipo rápido: no necesito demasiado pre-procesado
- Dominios complejos donde no existe una clara separación lineal en los datos
- Si tengo que predecir rápido (≈ log(n_train))
- Tengo variables categóricas y numéricas
- Cuando tengo más de dos clases, o múltiples etiquetas!

# Representación del árbol

- Cada nodo especifica una prueba contra algún atributo (variable de entrada)
- Cada rama descendente está relacionada con algún valor posible de dicho atributo
- Cada nodo hoja va a tener una etiqueta o clase determinada

# Ejemplo: ayudando a fisa a encontrar una estrella

In [3]:
import pandas as pd
df = pd.read_csv('data/astronomia.csv', sep=';')
df

,dia,nubosidad,temperatura,humedad,viento,estrellas_visibles
0,1,soleado,caluroso,alta,leve,no
1,2,soleado,caluroso,alta,fuerte,no
2,3,nublado,caluroso,alta,leve,si
3,4,lluvia,templado,alta,leve,si
4,5,lluvia,frio,normal,leve,si
5,6,lluvia,frio,normal,fuerte,no
6,7,nublado,frio,normal,fuerte,si
7,8,soleado,templado,alta,leve,no
8,9,soleado,frio,normal,leve,si
9,10,lluvia,templado,normal,leve,si


![](files/images/decision_tree_1.PNG)

### ¿Cómo clasificamos la siguiente instancia?:

nubosidad = soleado

temperatura = alta

humedad = alta

viento = fuerte

# Pero, ¿árbol o conjunto de reglas?

** En realidad un árbol es una disyunción de conjunciones! **

- Un camino del árbol es una ** conjunción** de pruebas sobre distintos atributos

- El árbol es una ** disyunción ** de esas conjunciones

## Ergo, un árbol es un conjunto de reglas:

(nubosidad=soleado AND humedad=normal) OR (nubosidad=nublado) OR (nubosidad=lluvia AND viento=leve)

# Conjunto completo de reglas

- R1: Si (nubosidad=soleado) Y (humedad=alta) entonces estrellas_visibles=No
- R2: Si (nubosidad=soleado) Y (humedad=normal) entonces estrellas_visibles=Yes
- R3: Si (nubosidad=nublado) entonces estrellas_visibles=Yes
- R4: Si (nubosidad=lluvia) Y (viento=fuerte) entonces estrellas_visibles=No
- R5: Si (nubosidad=lluvia) Y (viento=leve) entonces estrellas_visibles=Yes

# Aprendiendo un árbol ...

Existen muchos algoritmos distintos (CLS, ID3, C4.5, ID4, ID5, C4.8, C5.0)

# Algoritmo básico: ID3 (Iterative Dicotomiser)

- Búsqueda voraz por el espacio de todos los árboles de clasificación posibles
- Enfoque top-down (el árbol se constuye desde la raíz hacia las hojas)
- Para cada nodo se pregunta: qué atributo debería ser utilizado en este momento?
- Crea una rama por cada valor posible de ese atributo
- Se repite el proceso sacando del conjunto de atributos los que ya se utilizaron anteriormente en esa rama
- Termina cuando todas las instancias pertenecen a la misma clase o todos los atributos fueron utilizados
- Se etiqueta los nodos hoja usando la mayoría de las instancias que caen dentro de ese camino

In [6]:
df

,dia,nubosidad,temperatura,humedad,viento,estrellas_visibles
0,1,soleado,caluroso,alta,leve,no
1,2,soleado,caluroso,alta,fuerte,no
2,3,nublado,caluroso,alta,leve,si
3,4,lluvia,templado,alta,leve,si
4,5,lluvia,frio,normal,leve,si
5,6,lluvia,frio,normal,fuerte,no
6,7,nublado,frio,normal,fuerte,si
7,8,soleado,templado,alta,leve,no
8,9,soleado,frio,normal,leve,si
9,10,lluvia,templado,normal,leve,si


In [5]:
# Obtener el mejor atributo para el nodo raíz.
best_attribute = get_best_attribute_to_test(attributes=dataset.columns_names, data=train_data)

# Generar el nodo raíz con las ramas y los nodos a crear (si fuese necesario).
nodes = tree.add_node(Node(attribute=best_attribute, 
                           remaining_attributes=dataset.columns_names,
                           dataset[best_attribute]))

to_process.extend(nodes)

while to_process:
    
    current_node = to_process.pop()
    
    # Verifico si es un nodo hoja.
    if(len(set(current_node.class_labels) > 1) or (len(current_node.remaining_attributes) > 0)):
        # Determinar el mejor atributo dentro de los que no fueron evaluados en ese camino.
        best_attribute = get_best_attribute_to_test(attributes=current_node.remaining_attributes,
                                                    data=train_data)

        # Generar las ramas y los nodos a crear (si fuese necesario).
        nodes = tree.add_node(Node(attribute=best_attribute, 
                                   remaining_attributes=current_node.remaining_attributes,
                                   dataset[best_attribute].unique()))

        to_process.extend(nodes)
    else:
        tree.add_leaf(current_node.class_labels[0])

![](files/images/decision_tree_1.PNG)

## ¿Cómo elegir cuál es el mejor atributo en cada caso?

Queremos elegir el atributo que haga que los datos se separen lo mejor posible !

Para eso usamos el concepto de Información Mutua:

$$ I(C, X_i) = H(C) - H(C | X_i) $$

donde

$$ H(C) = - \sum_{c} p(c) \log_2 p(c) $$
$$ H(C|X)= - \sum_{c} \sum_{x} p(x,c) \log_2 p(c|x) $$

![](files/images/meme_dog.gif)

In [4]:
df

,dia,nubosidad,temperatura,humedad,viento,estrellas_visibles
0,1,soleado,caluroso,alta,leve,no
1,2,soleado,caluroso,alta,fuerte,no
2,3,nublado,caluroso,alta,leve,si
3,4,lluvia,templado,alta,leve,si
4,5,lluvia,frio,normal,leve,si
5,6,lluvia,frio,normal,fuerte,no
6,7,nublado,frio,normal,fuerte,si
7,8,soleado,templado,alta,leve,no
8,9,soleado,frio,normal,leve,si
9,10,lluvia,templado,normal,leve,si


# Con qué atributo (variable) empezamos ?

## Viento ?

$$ H(C) = -p(si) \log_2 p(si) - p(no) \log_2 p(no) =  -\frac{9}{14} \log_2 \frac{9}{14} - \frac{5}{14} \log_2 \frac{5}{14} = 0.94 $$

$$ H(C|Viento) = -p(fuerte, si) \log_2 p(si|fuerte) - p(fuerte, no) \log_2 p(no|fuerte) - p(leve, si) \log_2 p(si|leve) - p(debil, no) log_2 p(no|debil) = $$
$$ - \frac{3}{14} \log_2 \frac{3}{6} - \frac{3}{14} \log_2 \frac{3}{6} - \frac{6}{14} \log_2 \frac{6}{8} - \frac{2}{14} \log_2 \frac{2}{8} = 0.892 $$

I(C, Viento) = 0.94 - 0.892 = 0.048

## De la misma forma,

I(C, Humedad) = 0.151

** I(C, Nubosidad) = 0.246 **

I(C, Temperatura) = 0.029

![](files/images/ID3_1.PNG)

![](files/images/ID3_2.PNG)

Si el valor de **nubosidad** es "nublado", todas las instancias son positivas!. 

Lo transformamos entonces en un nodo hoja, al que le asignamos esa misma etiqueta para clasificar.

## Algunas observaciones ...

- El algoritmo no asegura encontrar el óptimo global
- La complexidad se incrementa de forma lineal con el número de instancias de entrenamiento y de forma exponencial en relación al número de atributos
- Variables con mayor cantidad de valores son favorecidas en la selección

# Mejoras posibles ...

- ** Establecer una profundidad máxima para el árbol **
- Manejo de variables continuas
- Permitir valores nulos en los datos de entrada
- Definir pesos a las clases o etiquetas (problemas cost-sensitive)
- Post-prunning

# Algunas desventajas ...

- Cuidado: overfitting !
- Inestables: pequeños cambios en los datos pueden hacer que el árbol resultante cambie notablemente
- Encontrar el árbol óptimo es un problema NP-completo
- Hay que tener cuidado si el dataset no está balanceado

# Tips y consejos prácticos ...

- A mayor cantidad de variables aumentan las probabilidades de sobre-entrenamiento: usar max_depth y analizar las componentes de nuestro conjunto de datos (técnicas de reducción de dimensionalidad).
- Analizar los árboles resultantes: dan mucha información de qué es lo que está pasando (FSS) !.
- Balancear los datos.
- Puedo usarlos para hacer análisis post-mortem en otros problemas.
- Uno de los algoritmos más utilizados: Classification And Regression Trees, o CART (permite trabajar con problemas de regresión obteniendo el promedio de las instancias que quedan en cada hoja)

![](files/images/Classifiers_comparison.png)